In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
apple_app=pd.read_csv("Resources/AppleStore.csv")
apple_app.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


In [3]:
android_app=pd.read_csv("Resources/googleplaystore.csv")
android_app.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
apple_app.dtypes

Unnamed: 0            int64
id                    int64
track_name           object
size_bytes            int64
currency             object
price               float64
rating_count_tot      int64
rating_count_ver      int64
user_rating         float64
user_rating_ver     float64
ver                  object
cont_rating          object
prime_genre          object
sup_devices.num       int64
ipadSc_urls.num       int64
lang.num              int64
vpp_lic               int64
dtype: object

In [5]:
android_app.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

# Transform the Android Table

In [6]:
android=android_app[['App','Reviews','Rating','Genres']]
android.head()

,App,Reviews,Rating,Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,159,4.1,Art & Design
1,Coloring book moana,967,3.9,Art & Design;Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",87510,4.7,Art & Design
3,Sketch - Draw & Paint,215644,4.5,Art & Design
4,Pixel Draw - Number Art Coloring Book,967,4.3,Art & Design;Creativity


In [7]:
android=android.dropna()

In [8]:
#Sort by the name, and observed that there are duplicated apps being rated multiple times i.e same names appears multiple times with the same or different reviews and ratings
android=android.sort_values("App")
#Trim the spaces of all the columns, covert all columns to a string datatype at this point
android['App']=android['App'].str.strip()
android['Reviews']=android['Reviews'].str.strip()
android['Rating']=android['Rating'].astype(str).str.strip()
android['Genres']=android['Genres'].str.strip()
#Change the app name case to a lower case prepare for the further duplicates removal
android['App']=android['App'].str.lower()

In [9]:
#The methodology is if there are duplicated names, we want to take the best reviews and ratings as the result of this app 
#so we created two extra columns and groupby app names to store the max review and ratings
android[['Reviews_max', 'Ratings_max']]=android.groupby('App')['Reviews', 'Rating'].transform('max')
android.head(5)

,App,Reviews,Rating,Genres,Reviews_max,Ratings_max
8532,+download 4 instagram twitter,40467,4.5,Social,40467,4.5
324,- free comics - comic apps,115,3.5,Comics,115,3.5
4541,.r,259,4.5,Tools,259,4.5
4636,/u/app,573,4.7,Communication,573,4.7
5940,058.ba,27,4.4,News & Magazines,27,4.4


In [10]:
#and then we use the max review and ratings as the new review columns and rating columns and prepare for the duplicate removal
new_android=android[['App','Reviews_max','Ratings_max','Genres']]
new_android.head(5)

,App,Reviews_max,Ratings_max,Genres
8532,+download 4 instagram twitter,40467,4.5,Social
324,- free comics - comic apps,115,3.5,Comics
4541,.r,259,4.5,Tools
4636,/u/app,573,4.7,Communication
5940,058.ba,27,4.4,News & Magazines


In [11]:
android_table=new_android.drop_duplicates().reset_index(drop=True)
android_table.head(5)

,App,Reviews_max,Ratings_max,Genres
0,+download 4 instagram twitter,40467,4.5,Social
1,- free comics - comic apps,115,3.5,Comics
2,.r,259,4.5,Tools
3,/u/app,573,4.7,Communication
4,058.ba,27,4.4,News & Magazines


In [12]:
android_table=android_table.rename(columns={"App":"App Name","Reviews_max":"Total_Reviews","Ratings_max":"Ratings"})

In [13]:
android_table.head(5)

,App Name,Total_Reviews,Ratings,Genres
0,+download 4 instagram twitter,40467,4.5,Social
1,- free comics - comic apps,115,3.5,Comics
2,.r,259,4.5,Tools
3,/u/app,573,4.7,Communication
4,058.ba,27,4.4,News & Magazines


In [14]:
#Convert the numerical fields to a number and found the 3.0M gave us an error
android_table['Ratings']=pd.to_numeric(android_table['Ratings'])
android_table['Total_Reviews']=pd.to_numeric(android_table['Total_Reviews'])

ValueError: Unable to parse string "3.0M" at position 4775

In [15]:
#To fix that, we need to change 3.0M to 3,000,000
android_table.loc[android_table.Total_Reviews=='3.0M','Total_Reviews']=3000000

In [16]:
#now we are happy
android_table['Total_Reviews']=pd.to_numeric(android_table['Total_Reviews'])

In [17]:
#Check the data type, it looks awesome!
android_table.dtypes

App Name          object
Total_Reviews      int64
Ratings          float64
Genres            object
dtype: object

# Transform the Apple table

In [18]:
apple=apple_app[['track_name','rating_count_tot','user_rating','prime_genre']]
apple.head()

,track_name,rating_count_tot,user_rating,prime_genre
0,PAC-MAN Premium,21292,4.0,Games
1,Evernote - stay organized,161065,4.0,Productivity
2,"WeatherBug - Local Weather, Radar, Maps, Alerts",188583,3.5,Weather
3,"eBay: Best App to Buy, Sell, Save! Online Shop...",262241,4.0,Shopping
4,Bible,985920,4.5,Reference


In [19]:
#drop n/a before convert everything to a string
apple=apple.dropna()

In [20]:
#Firstly trim the spaces of all columns
apple['track_name']=apple['track_name'].str.strip()
apple['rating_count_tot']=apple['rating_count_tot'].astype(str).str.strip()
apple['user_rating']=apple['user_rating'].astype(str).str.strip()
apple['prime_genre']=apple['prime_genre'].str.strip()

In [21]:
#Convert the numerical fields back
apple['rating_count_tot']=pd.to_numeric(apple['rating_count_tot'])
apple['user_rating']=pd.to_numeric(apple['user_rating'])

In [22]:
apple.dtypes

track_name           object
rating_count_tot      int64
user_rating         float64
prime_genre          object
dtype: object

In [23]:
#Change the name column to a lower case and sort by it to prepare for removing the duplicates, same methodology as the Android one 
apple['track_name']=apple['track_name'].str.lower()
apple=apple.sort_values('track_name')
apple[['Reviews_max', 'Ratings_max']]=apple.groupby('track_name')['rating_count_tot', 'user_rating'].transform('max')

In [24]:
#Take the review_max and ratings_max column as the new Ratings and Total_Reviews column and then remove the duplicates
new_apple=apple[['track_name','Reviews_max', 'Ratings_max','prime_genre']]
new_apple=new_apple.drop_duplicates().reset_index(drop=True)

In [25]:
#rename the field names to match Android table
apple_table=new_apple.rename(columns={"track_name":"App Name","Reviews_max":"Total_Reviews","Ratings_max":"Ratings","prime_genre":"Genres"})



# Merge the two table to one table

In [26]:
apple_table.head()

,App Name,Total_Reviews,Ratings,Genres
0,! oh fantastic free kick + kick wall challenge,0,0.0,Games
1,"""burn your fat with me!!""",302,4.5,Health & Fitness
2,"""hook""",959,5.0,Games
3,"""klocki""",587,4.5,Games
4,( offtime ) light – track how much you use you...,22,2.0,Health & Fitness


In [27]:
android_table.head()

,App Name,Total_Reviews,Ratings,Genres
0,+download 4 instagram twitter,40467,4.5,Social
1,- free comics - comic apps,115,3.5,Comics
2,.r,259,4.5,Tools
3,/u/app,573,4.7,Communication
4,058.ba,27,4.4,News & Magazines


In [28]:
#merge apple table and android table
app_overall = pd.merge(android_table, apple_table, on='App Name', how='inner')#suffixes=("_Android","_Apple")

In [29]:
app_overall.head()

,App Name,Total_Reviews_x,Ratings_x,Genres_x,Total_Reviews_y,Ratings_y,Genres_y
0,adp mobile solutions,85185,4.3,Business,8324,4.0,Business
1,aj jump: animal jam kangaroos!,2975,4.4,Arcade,834,4.5,Games
2,amc,20843,3.2,Entertainment,3105,2.0,Entertainment
3,asos,181823,4.7,Shopping,9725,5.0,Shopping
4,adobe illustrator draw,65766,4.4,Photography,3215,4.5,Productivity


In [30]:
app_overall.columns

Index(['App Name', 'Total_Reviews_x', 'Ratings_x', 'Genres_x',
       'Total_Reviews_y', 'Ratings_y', 'Genres_y'],
      dtype='object')

In [31]:
app_overall.dtypes

App Name            object
Total_Reviews_x      int64
Ratings_x          float64
Genres_x            object
Total_Reviews_y      int64
Ratings_y          float64
Genres_y            object
dtype: object

In [32]:
#taking the total of the app store rating and the android reviews
Reviews_df = app_overall["Total_Reviews_x"]+app_overall["Total_Reviews_y"]

In [33]:
#taking mean of ratings from apple store and google store
Ratings_df = app_overall[['Ratings_x', 'Ratings_y']].mean(axis=1)

In [34]:
#add a column for best reviews 
app_overall['Total_Reviews'] = Reviews_df

In [35]:
#add a column for best rating
app_overall['Ratings'] =Ratings_df

In [36]:
app_overall.head()

,App Name,Total_Reviews_x,Ratings_x,Genres_x,Total_Reviews_y,Ratings_y,Genres_y,Total_Reviews,Ratings
0,adp mobile solutions,85185,4.3,Business,8324,4.0,Business,93509,4.15
1,aj jump: animal jam kangaroos!,2975,4.4,Arcade,834,4.5,Games,3809,4.45
2,amc,20843,3.2,Entertainment,3105,2.0,Entertainment,23948,2.60
3,asos,181823,4.7,Shopping,9725,5.0,Shopping,191548,4.85
4,adobe illustrator draw,65766,4.4,Photography,3215,4.5,Productivity,68981,4.45


In [37]:
clean_tab = app_overall[['App Name','Genres_x','Total_Reviews','Ratings']]

In [38]:
clean_tab.head()

,App Name,Genres_x,Total_Reviews,Ratings
0,adp mobile solutions,Business,93509,4.15
1,aj jump: animal jam kangaroos!,Arcade,3809,4.45
2,amc,Entertainment,23948,2.60
3,asos,Shopping,191548,4.85
4,adobe illustrator draw,Photography,68981,4.45


In [39]:
#rename the column for genress 
final=clean_tab.rename(columns={"Genres_x":"Genres","App Name":"App_Name"})

In [40]:
#Sort by the ratings and then total reviews
final=final.sort_values(["Total_Reviews","Ratings"],ascending=False).reset_index(drop=True)

In [41]:
final['App_Name']=final['App_Name'].str.title()
final.head()

,App_Name,Genres,Total_Reviews,Ratings
0,Facebook,Social,81132982,3.80
1,Whatsapp Messenger,Communication,69406905,4.45
2,Instagram,Social,68739004,4.50
3,Clash Of Clans,Strategy,47024693,4.55
4,Subway Surfers,Arcade,28431462,4.50


# Web Scrap on an article to capture the top 20 downloaded apps and create another table

In [42]:
#To showcase more of our skills, at the very last moninute we decided to web scarp a articale that has the top 20 downloaded apps and then load into another table
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [43]:
url = 'https://mashable.com/article/apple-most-popular-iphone-apps-2018'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html data-env="production" lang="en" xml:lang="en">
 <head>
  <script>
   window.__o = {"channel":"tech","content_type":"article","v_buy_i":null,"h_pub_buy":null,"v_cur_i":0,"v_max_i":0,"events":"","top_channel":"tech","content_source_type":"Internal - Editorial Series","content_source_name":"none","author_name":"Karissa Bell","age":"134","pub_day":4,"pub_month":12,"pub_year":2018,"pub_date":"12/04/2018","sourced_from":"Internal","isPostView":true,"post_lead_type":"Alt Image Lead","topics":"app-store,apple,apps-and-software,consumer-tech,end-of-year-2018,iphone,tech,youtube","campaign":null,"display_mode":null,"viral_video_type":null,"standalone_video_show":null,"mash_video_show":null,"b_flag":true,"cms_author_name":null,"cms_author_geo":"us","cms_author_team":"Technology","bp_id":84918,"shoppable":null,"voice":"Mashable","level_of_effort":"Less than one hour","content_format":"Article","video_platform_target":null,"key_topics":["Consumer Tech"],"job_function":"News Hi

In [44]:
results = soup.find_all('ol')[0].find_all('li')
#Create a blank list
appnames=[]
for result in results:
    title = result.text
    appnames.append(title)

In [45]:
#Convert the result to a dataframe
dic={"Top_Apps":appnames}
topapp=pd.DataFrame(dic)
topapp

,Top_Apps
0,YouTube
1,Instagram
2,Snapchat
3,Messenger
4,Facebook
5,Bitmoji
6,Netflix
7,Google Maps
8,Gmail
9,Spotify Music


# Connect to local database

In [46]:
import pymysql
pymysql.install_as_MySQLdb()
rds_connection_string = "root:MYdd2012@127.0.0.1:3306/PopApp"
engine = create_engine(f'mysql://{rds_connection_string}')

In [47]:
#Load two dataframes to the SQL tables 
final.to_sql(name='Applications', con=engine, if_exists='append', index=False)
topapp.to_sql(name='PopularApps', con=engine, if_exists='append', index=False)

C:\Users\luji5563\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'Applications' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\Users\luji5563\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1144: UserWarning: The provided table name 'PopularApps' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [48]:
#Query to see if the table is successfully created
pd.read_sql_query('select * from Applications', con=engine).head()

,App_Name,Genres,Total_Reviews,Ratings
0,Facebook,Social,81132982,3.80
1,Whatsapp Messenger,Communication,69406905,4.45
2,Instagram,Social,68739004,4.50
3,Clash Of Clans,Strategy,47024693,4.55
4,Subway Surfers,Arcade,28431462,4.50


In [49]:
pd.read_sql_query('select * from PopularApps', con=engine).head()

,Top_Apps
0,YouTube
1,Instagram
2,Snapchat
3,Messenger
4,Facebook


In [50]:
# See which apps in our dataset actually appears on the top 20 apps article
pd.read_sql_query('select * from applications inner join popularapps on applications.App_Name=popularapps.Top_Apps', con=engine).head()

,App_Name,Genres,Total_Reviews,Ratings,Top_Apps
0,Facebook,Social,81132982,3.80,Facebook
1,Whatsapp Messenger,Communication,69406905,4.45,WhatsApp Messenger
2,Instagram,Social,68739004,4.50,Instagram
3,Snapchat,Social,17339257,3.25,Snapchat
4,Twitter,News & Magazines,12021461,3.90,Twitter


# Thank you for grading this!